Ex.2) We will now consider another general formulation of the form given below:
min c
⊤x,
s.t Ax ≤ b
l ≤ x ≤ u.
where c, x, l and u are N × 1 vectors and c
⊤x =
PN
i=1 cixi
. A is a M × N matrix and b is M × 1
vector. Ax ≤ b results in M constraints and note that i-th row of Ax ≤ b can be written as a
constraint of the form:
ai1x1 + ai2x2 + . . . + aiN xN ≤ bi
.
We will solve the general form using concrete values given below:
c = [0.56, 1.721, −9.6, −2.85, 41.5, 33.05, −0.75, 20.5, −0.35, −31.02, 50.65, 6.125];
A =


1 −1 0 0 2 −2 1 0 0 −1 1 −2
1 0 0 −1 0 −2 0 −1 1 0 0 −1
−1 0 0 1 −1 2.5 −3.5 0 2 −1 1 0
0 1 −0.45 0 −2.78 0 −3.4 0 −1 1 −1 0.25
0 −1 1 0 1 0 1 3 −2 0 1 0


b = [88.5, 35.4, 47.9, 41.43, 22.7]
l = [0, 0, −∞, −∞, −34, 0, −67, 0, −23, −5, 6, 1],
u = [12, 503, 4, +∞, +∞, 2, 20, 55, 23, 15, +∞, +∞]

In [37]:
!pip install -q pyomo

In [38]:
from pyomo.environ import *

In [39]:
import numpy as np

In [40]:
model = ConcreteModel()

In [41]:
N = 12
M = 5

In [42]:
obj_coef_c = np.array([0.56,1.721,-9.6,-2.85,41.5,33.05,-0.75,20.5,-0.35,-31.02,50.65,6.125])

In [43]:
constr_coef_A = np.array([[1,-1,0,0,2,-2,1,0,0,-1,1,-2],[1,0,0,-1,0,-2,0,-1,1,0,0,-1],[-1,0,0,1,-1,2.5,-3.5,0,2,-1,1,0],[0,1,-0.45,0,-2.78,0,-3.4,0,-1,1,-1,0.25],[0,-1,1,0,1,0,1,3,-2,0,1,0]])
constr_rhs_b = np.array([88.5,35.4,47.9,41.43,22.7])

In [44]:
lower_bound = np.array([0,0,-np.inf,-np.inf,-34,0,-67,0,-23,-5,6,1])
upper_bound = np.array([12,503,4,np.inf,np.inf,2,20,55,23,15,np.inf,np.inf])

In [45]:
col_indices = np.arange(N)
row_indices = np.arange(M)
print(col_indices)
print(col_indices[11])

[ 0  1  2  3  4  5  6  7  8  9 10 11]
11


In [46]:
model.x = Var(col_indices)

In [47]:
model.constraints = ConstraintList()

In [48]:
for i in row_indices:
  model.constraints.add(sum(constr_coef_A[i][j]*model.x[j] for j in col_indices) <= constr_rhs_b[i])
  

  

In [49]:
for j in col_indices:
  model.x[j].setlb(lower_bound[j])
  model.x[j].setub(upper_bound[j])

In [50]:
model.objective = Objective(expr = sum(obj_coef_c[j]*model.x[j] for j in col_indices), sense=minimize)

In [51]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   0.0 :  None :  12.0 : False :  True :  Reals
          1 :   0.0 :  None : 503.0 : False :  True :  Reals
          2 :  None :  None :   4.0 : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
          4 : -34.0 :  None :  None : False :  True :  Reals
          5 :   0.0 :  None :   2.0 : False :  True :  Reals
          6 : -67.0 :  None :  20.0 : False :  True :  Reals
          7 :   0.0 :  None :  55.0 : False :  True :  Reals
          8 : -23.0 :  None :  23.0 : False 

In [52]:
!apt-get install -y -qq glpk-utils

In [53]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -1951.831
  Upper bound: -1951.831
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 13
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01276540756225586
# ----------------------------------------------------------
#   Solution Information
# ---------------------------

In [54]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()


Objective =  -1951.8309999999997

Decision Variables
x[ 0 ] =  12.0
x[ 1 ] =  0.0
x[ 2 ] =  4.0
x[ 3 ] =  119.82
x[ 4 ] =  -34.0
x[ 5 ] =  0.0
x[ 6 ] =  20.0
x[ 7 ] =  0.0
x[ 8 ] =  -7.46000000000002
x[ 9 ] =  15.0
x[ 10 ] =  6.0
x[ 11 ] =  1.0

Constraints
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :               -47.0 :  88.5
      2 :  None : -116.28000000000002 :  35.4
      3 :  None :   47.89999999999995 :  47.9
      4 :  None :   41.43000000000002 : 41.43
      5 :  None :   10.92000000000004 :  22.7


Q2.1) Optimal value of Objective function =  -1951.8309999999997

 Optimal value of Decision Variables
x[ 0 ] =  12.0
x[ 1 ] =  0.0
x[ 2 ] =  4.0
x[ 3 ] =  119.82
x[ 4 ] =  -34.0
x[ 5 ] =  0.0
x[ 6 ] =  20.0
x[ 7 ] =  0.0
x[ 8 ] =  -7.46000000000002
x[ 9 ] =  15.0
x[ 10 ] =  6.0
x[ 11 ] =  1.0

Q2.2)Constraints 3 and 4 are active because value is equal to upper value (i.e.LHS = RHS).and constraints 1, 2 and 5 are inactive because value is not equal to upper value(LHS != RHS).